# Run from jupyter

This page give an overview of the details of running a jupyter kernel from the jupyter notebook. It doesn't have much practical sence but extremely usefull for purposes of this project.

In [1]:
import sys
sys.path.append("/tmp")
import multiprocessing
from ipykernel.kernelbase import Kernel
from ipykernel.kernelapp import IPKernelApp
from jupyter_client.blocking import BlockingKernelClient

## Create new instance

The main problem associated with running kernel from jupyter environment - is that in execution environment already created kernel object. It follows singleton pattern which means that it has one instance over whole program, so if you try to create a new kernel you'll just get the same object - not create a new one. This can be overcome by running the kernel in the new process.

**Note** The process must be started in `spawn` or `forkserver` context so that it not to share resources with the parent process. For more details on this issue, see description on the different startup methods in the `multiprocessing` package. 

---

The following cell shows the instance of the `Kernel` class.

In [11]:
Kernel.instance()

As the result we got reference to the `IPythonKernel` - this is the kernel of the environment this notebook runned in.

So if you directly call `IPKernelApp.launch_instance` directly, you will just get error related to the fact that resources the kernel is trying to use are occupied, as shown in the next cell.

In [7]:
try:
    IPKernelApp.launch_instance()
except Exception as e:
    print(e)

init_sockets cannot be called twice!


The next code creates prints `Kernel.instance()` from the separate process.

In [ ]:
%%writefile /tmp/show_kernel_instance.py
from ipykernel.kernelbase import Kernel

def process_target():
    print(Kernel.instance())

Overwriting /tmp/show_kernel_instance.py


In [12]:
from show_kernel_instance import process_target
context = multiprocessing.get_context("spawn")
p = context.Process(target=process_target)
p.start()

In the separate process, `Kernel.instance` refers to the different object.

## Solution

In [2]:
%%writefile /tmp/run_kernel.py
from ipykernel.kernelapp import IPKernelApp

def process_target():
    IPKernelApp.launch_instance(["-f", "/tmp/example_connection.json"])

Overwriting /tmp/run_kernel.py


In [6]:
import sys 
sys.path.append("/tmp")
from run_kernel import process_target

context = multiprocessing.get_context("spawn")
p = context.Process(target=process_target)
p.start()

/home/fedor/Documents/python/packages/jupyter/kernel
NOTE: When using the `ipython kernel` entry point, Ctrl-C will not work.

To exit, you will have to explicitly quit this process, by either sending
"quit" from a client, or using Ctrl-\ in UNIX-like environments.

To read more about this, see https://github.com/ipython/ipython/issues/2049


To connect another client to this kernel, use:
    --existing /tmp/example_connection.json


In [7]:
client = BlockingKernelClient()
client.load_connection_file("/tmp/example_connection.json")
client.start_channels()
client.is_alive()

True